### Get length of video file

In [ ]:
# Find the first JS log file in the directory and find out how long the video is. I assume that all log data in this entire directory has the same length of video.
import os
import re
SEC_IN_MIN = 60
seg_lens = ["5s", "10s", "20s"]
vid_length_pattern = re.compile("^LOG  [\d\.]+,([\d\.]+):.+$", re.M)

video_length_sec = None

for seg_len in seg_lens:
    dir_path = "./{seg_len}".format(seg_len = seg_len)
    
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)
        if not os.path.isdir(file_path): # Only looking to read directories (that are symlinks).
            continue
        jsLog_names = [filename for filename in os.listdir(file_path) if filename.startswith("Tester_") and filename.endswith(".log")]
        if len(jsLog_names) != 1:
            continue
        jsLog_name = jsLog_names[0]
        
        last_line = None
        with open(os.path.join(file_path, jsLog_name), "r") as jsLog:
            for line in jsLog:
                pass
            last_line = line
        
        match = vid_length_pattern.match(last_line)
        if not match:
            continue
        video_length_sec = match.group(1)
        break
        
    if video_length_sec is not None:
        break

video_length_sec = float(video_length_sec)
print("Video length (sec): {vidlen}".format(vidlen = video_length_sec))
video_length_min = video_length_sec / SEC_IN_MIN

: 

In [ ]:
import os
import csv
import numpy as np

def find_missing_and_out_of_order_values(data):
    missing_values = []
    out_of_order_values = []

    for i, value in enumerate(data):
        # Check if the current value is missing
        if i not in data:
            missing_values.append(i)

        # Check if the current value is out of order
        if i > 0 and value < data[i-1]:
            out_of_order_values.append(data[i-1])
            out_of_order_values.append(value)

    return missing_values, out_of_order_values

def count_elements_layered_list(nested_list):
    count = 0
    for element in nested_list:
        if isinstance(element, list):
            count += count_elements_layered_list(element)
        else:
            count += 1
    return count

def calculate_rtt(folder_path):
    folder_path = folder_path
    run_folders = ["Run0", "Run1", "Run2"]
    all_packet_loss_seqs=[]
    all_missing_seqs=[]
    all_out_of_order_seqs=[]
    rtt_list = []
    receive_rates =[]
    sends=[]

    for run_folder in run_folders:
        file_path = os.path.join(folder_path, run_folder, "UDPing_log.csv")
        with open(file_path, "r") as csv_file:
            csv_reader = csv.reader(csv_file)
            next(csv_reader) # skip "Pinging..." row
            next(csv_reader) # skip "empty" row
            next(csv_reader) # skip header row
            for row in csv_reader:
                if "Ping statistics" in row[0]:
                    break
                rtt = float(row[2])
                rtt_list.append(rtt)

            send=int(csv_reader.__next__()[1])
            receive=int(csv_reader.__next__()[1])
            sends.append(send)
            if receive == 0:
                receive_rates.append(0) # There are cases that all packets got lost.
            else:
                receive_rates.append(send/receive)


        #get the seq that is not one above the previous seq
        with open(file_path, "r") as csv_file:
            csv_reader = csv.reader(csv_file)
            prev_seq=-9999
            packet_loss_seqs=[]
            next(csv_reader)  # skip "Pinging..." row
            next(csv_reader)  # skip "empty" row
            next(csv_reader)  # skip header row
            for row in csv_reader:
                if "Ping statistics" in row[0]:
                    break

                seq = int(row[3])
                if prev_seq != -9999 and seq != prev_seq + 1:
                    packet_loss_seqs.append(seq)
                prev_seq = seq

            print(packet_loss_seqs)
            all_packet_loss_seqs.append(packet_loss_seqs)

        #Find missing and out of order values
        with open(file_path, "r") as csv_file:
            seq_column = []
            csv_reader = csv.reader(csv_file)
            next(csv_reader)  # skip "Pinging..." row
            next(csv_reader)  # skip "empty" row
            next(csv_reader)  # skip header row
            for row in csv_reader:
                if "Ping statistics" in row[0]:
                    break
                seq_column.append(int(row[3]))
            missing_values, out_of_order_values=find_missing_and_out_of_order_values(seq_column)
            all_missing_seqs.append(missing_values)
            all_out_of_order_seqs.append(out_of_order_values)

    mean_rtt = np.mean(rtt_list)
    std_rtt = np.std(rtt_list)
    mean_receive_rate=np.mean(receive_rates)
    std_receive_rate=np.std(receive_rates)

    #calculate missing rate and out-of-order rate
    missing_seqs_count=count_elements_layered_list(all_missing_seqs)
    missing_seqs_rate=missing_seqs_count/sum(sends)

    out_of_order_count=count_elements_layered_list(all_out_of_order_seqs)
    out_of_order_rate=out_of_order_count/sum(sends)



    return (mean_rtt, std_rtt,mean_receive_rate,std_receive_rate, all_packet_loss_seqs, all_missing_seqs, all_out_of_order_seqs, sends,missing_seqs_rate,out_of_order_rate)

mean_rtt, std_rtt, mean_receive_rate, std_receive_rate, packet_loss_seqs, missing_seqs, out_of_order_seqs, sends,missing_seqs_rate,out_of_order_rate= calculate_rtt(".\\5s")
print("Mean RTT:", mean_rtt)
print("Standard deviation of RTT:", std_rtt)
print("Mean Receive rate:", mean_receive_rate)
print("Standard deviation of Receive rate:", std_receive_rate)
print("Packet loss sequences:", packet_loss_seqs)
print("Packet loss sequences count:", count_elements_layered_list(packet_loss_seqs))
print("Missing sequences:",missing_seqs)
print("Missing sequences count:", count_elements_layered_list(missing_seqs))
print("Missing sequences rate:", missing_seqs_rate)
print("Out of order sequences:",out_of_order_seqs)
print("Out of order sequences count:",count_elements_layered_list(out_of_order_seqs))
print("Out of order rate:",out_of_order_rate)

: 

In [ ]:

#RTT graph
#missing sequences drawed in certain time intervals
#missing packets connect lines， if 1 packets missing, go to 1. If 2 packets missing, go to 2.


: 

In [ ]:
import pandas as pd
import os
import csv
import numpy as np
import plotly.graph_objects as go

# function to find the closest value in a series to a given value
def find_closest_value(series, value):
    return series.iloc[(series - value).abs().argsort()[0]]

def find_missing_and_out_of_order_values(data):
    missing_values = []
    out_of_order_values = []

    for i, value in enumerate(data):
        # Check if the current value is missing
        if i not in data:
            missing_values.append(i)

        # Check if the current value is out of order
        if i > 0 and value < data[i-1]:
            out_of_order_values.append(data[i-1])
            out_of_order_values.append(value)

    return missing_values, out_of_order_values

# Read the csv file and skip the first two rows
df = pd.read_csv('.\\20s\\Run2\\UDPing_log.csv', skiprows=[0, 1])

# Loop through each row in the dataframe
break_index = None
for index, row in df.iterrows():
    # Check if the row contains "Ping statistics" and nothing else
    if "Ping statistics" in row[0]:
        # If it does, break out of the loop
        break_index = index
        break

# Slice the dataframe to include only the rows up to the "Ping statistics" row
df = df.iloc[:break_index]
df = df.rename(columns={'tSent(ms) ': 'tSent(ms)'})
df['tSent(s)'] = df['tSent(ms)'] / 1000
df['tSent(min)'] = df['tSent(s)'] / 60
# create normal_seq column
df['normal_seq'] = df['seq'].diff().apply(lambda x: 1 if pd.isna(x) or x == 0 else 1 if x == 1 else 0)

# Create the missing_seq and out_of_order_seq columns
seq_in_list = df['seq'].tolist()
missing_values, out_of_order_values = find_missing_and_out_of_order_values(seq_in_list)

df['out_of_order_seq'] = df['seq'].apply(lambda x: 0 if x in out_of_order_values else 1)

# create a new column 'missing_seq'
df['missing_seq'] = np.nan

# loop over the missing values and find the closest value in the 'seq' column
for value in missing_values:
    closest_value = find_closest_value(df['seq'], value)
    # set the 'missing_seq' value to 0 for the closest value
    df.loc[df['seq'] == closest_value, 'missing_seq'] = 0

# fill any remaining missing values in the 'missing_seq' column with 1
df['missing_seq'].fillna(1, inplace=True)

# Print the resulting dataframe
print(df)

: 

: 

In [ ]:
df['missing_seq']

: 

In [ ]:
def set_std_plot_params(fig):
    fig.update_layout(font = dict(size = 24))
    # Remove the background coloring.
    fig.update_layout({"plot_bgcolor": "rgba(0,0,0,0)", "paper_bgcolor": "rgba(0,0,0,0)"})
    # Make the gridlines visible on the transparent background.
    fig.update_xaxes(showgrid = True, gridwidth = 1, gridcolor = "rgba(169,169,169,0.5)")
    fig.update_yaxes(showgrid = True, gridwidth = 1, gridcolor = "rgba(169,169,169,0.5)", tickvals=[0, 1])

# Create the scatter plot
fig = go.Figure()

# Add scatter traces for each column
for column in ['out_of_order_seq', 'missing_seq']:
    fig.add_trace(go.Scatter(
        x=df['tSent(min)'],
        y=df[column].apply(lambda x: "In Order" if x == 1 else "Out of Order"),
        mode='markers+lines',
        name="Out of Order Sequences" if column == 'out_of_order_seq' else "Missing Sequences",
        line=dict(color='red') if column == 'out_of_order_seq' else dict(color='blue'),
        line_shape='spline',
        line_smoothing=0.7,
        line_width=1.5,
        marker=dict(size=7)
))

# Update layout with axis labels
fig.update_layout(
    xaxis_title='Time(min)',
    yaxis_title='In Order / Out of Order',
    font=dict(size=18)
)

set_std_plot_params(fig)

# Show the plot
fig.show()



: 

In [ ]:
def produce_graph(df):
    # function to find the closest value in a series to a given value
    def find_closest_value(series, value):
        return series.iloc[(series - value).abs().argsort()[0]]

    def find_missing_and_out_of_order_values(data):
        missing_values = []
        out_of_order_values = []

        for i, value in enumerate(data):
            # Check if the current value is missing
            if i not in data:
                missing_values.append(i)

            # Check if the current value is out of order
            if i > 0 and value < data[i-1]:
                out_of_order_values.append(data[i-1])
                out_of_order_values.append(value)

        return missing_values, out_of_order_values

    # Read the csv file and skip the first two rows
    # df = pd.read_csv('.\\5s\\Run0\\UDPing_log.csv', skiprows=[0, 1])


    # Loop through each row in the dataframe
    break_index = None
    for index, row in df.iterrows():
        # Check if the row contains "Ping statistics" and nothing else
        if "Ping statistics" in row[0]:
            # If it does, break out of the loop
            break_index = index
            break

    # Slice the dataframe to include only the rows up to the "Ping statistics" row
    df = df.iloc[:break_index]
    df = df.rename(columns={'tSent(ms) ': 'tSent(ms)'})
    df['tSent(s)'] = df['tSent(ms)'] / 1000
    df['tSent(min)'] = df['tSent(s)'] / 60
    # create normal_seq column
    df['normal_seq'] = df['seq'].diff().apply(lambda x: 1 if pd.isna(x)
                                            or x == 0 else 1 if x == 1 else 0)

    # Create the missing_seq and out_of_order_seq columns
    seq_in_list = df['seq'].tolist()
    missing_values, out_of_order_values = find_missing_and_out_of_order_values(
        seq_in_list)

    df['out_of_order_seq'] = df['seq'].apply(
        lambda x: 0 if x in out_of_order_values else 1)

    # create a new column 'missing_seq'
    df['missing_seq'] = np.nan

    # loop over the missing values and find the closest value in the 'seq' column
    for value in missing_values:
        closest_value = find_closest_value(df['seq'], value)
        # set the 'missing_seq' value to 0 for the closest value
        df.loc[df['seq'] == closest_value, 'missing_seq'] = 0

    # fill any remaining missing values in the 'missing_seq' column with 1
    df['missing_seq'].fillna(1, inplace=True)


    def set_std_plot_params(fig):
        fig.update_layout(font=dict(size=24))
        # Remove the background coloring.
        fig.update_layout({"plot_bgcolor": "rgba(0,0,0,0)",
                        "paper_bgcolor": "rgba(0,0,0,0)"})
        # Make the gridlines visible on the transparent background.
        fig.update_xaxes(showgrid=True, gridwidth=1,
                        gridcolor="rgba(169,169,169,0.5)")
        fig.update_yaxes(showgrid=True, gridwidth=1,
                        gridcolor="rgba(169,169,169,0.5)", tickvals=[0, 1])


    # Create the scatter plot
    fig = go.Figure()

    # Add scatter traces for each column
    for column in ['out_of_order_seq', 'missing_seq']:
        fig.add_trace(go.Scatter(
            x=df['tSent(min)'],
            y=df[column],
            mode='markers+lines',
            name="Out of Order Sequences" if column == 'out_of_order_seq' else "Missing Sequences",
            line=dict(color='red') if column == 'out_of_order_seq' else dict(
                color='blue'),
            line_shape='spline',
            line_smoothing=0.7,
            line_width=1.5,
            marker=dict(size=7)
        ))

    # Update layout with axis labels
    fig.update_layout(
        xaxis_title='Time(min)',
        yaxis_title='In Order(1) / Out of Order(0)',
        font=dict(size=18)
    )

    set_std_plot_params(fig)

    # Get the current file path
    filepath = os.path.splitext(__file__)[0]

    # Save the plot as a PNG file using the file path as the name
    fig.write_image(filepath + '.png')


: 

In [ ]:
import glob

# Search for files named "UDPing_log.csv" in "Bola", "Dynamic", and "ThurAll" folders and their subfolders
file_paths = glob.glob('.\\{Bola,Dynamic,ThurAll}/**/UDPing_log.csv', recursive=True)

# Read all files found into a single DataFrame
for path in file_paths:
    df = pd.read_csv(path, skiprows=[0, 1])
    produce_graph(df)

: 

In [ ]:

import pandas as pd
import plotly.graph_objects as go


def visualize_missing_sequences(missing,send):
    x = list(range(send))
    y = [1] * send

    for i in missing:
        y[i] = 0

    fig = go.Figure(data=go.Scatter(x=x, y=y, mode='lines+markers', marker=dict(color='red')))

    return fig


: 

In [ ]:
visualize_missing_sequences(missing_seqs[0],sends[0]).show()

: 

In [ ]:
visualize_missing_sequences(missing_seqs[1],sends[1]).show()

: 

In [ ]:
visualize_missing_sequences(missing_seqs[2],sends[2]).show()

: 

## Stalls

: 

In [ ]:
import os
import re

stall_pattern = re.compile("^STALL  ([\d]+) ms and stopped at [\d\.]+ sec", re.M)

def ComputeStallData(dir_path):
    jsLog_names = [filename for filename in os.listdir(dir_path) if filename.startswith("Tester_") and filename.endswith(".log")]
    if len(jsLog_names) != 1:
        return
    jsLog_name = jsLog_names[0]
    jsLog = open(os.path.join(dir_path, jsLog_name), "r")
    
    stall_lengths_ms = list()
    
    # Read through all the lines.
    while line := jsLog.readline():
        match = stall_pattern.match(line)
        if match:
            stall_len_ms = int(match.group(1))
            stall_lengths_ms.append(stall_len_ms)
    
    return stall_lengths_ms

: 

In [ ]:
import numpy as np
import os
import scipy.stats as st

seg_lens = ["5s", "10s", "20s"]
conf_interval = 0.95

avg_stall_cts = list()
avg_stall_cts_conf = list()
stdev_stall_cts = list()

avg_stall_lengths = list()
avg_stall_lengths_conf = list()
stdev_stall_lengths = list()

for seg_len in seg_lens:
    dir_path = "./{seg_len}".format(seg_len = seg_len)
    
    seglen_aggregate_stall_lengths = list()
    seglen_aggregate_stall_counts = list()
    
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)
        if not os.path.isdir(file_path): # Only looking to read directories (that are symlinks).
            continue
        cur_stall_lengths_ms = ComputeStallData(file_path)
        seglen_aggregate_stall_lengths += cur_stall_lengths_ms
        seglen_aggregate_stall_counts.append(len(cur_stall_lengths_ms))
    
    avg_cts = np.mean(seglen_aggregate_stall_counts)
    avg_stall_cts.append(avg_cts)
    avg_stall_cts_conf.append(st.norm.interval(alpha = conf_interval, loc = avg_cts, scale = st.sem(seglen_aggregate_stall_counts)))
    stdev_stall_cts.append(np.std(seglen_aggregate_stall_counts))
    
    avg_lengths = np.mean(seglen_aggregate_stall_lengths)
    avg_stall_lengths.append(avg_lengths)
    avg_stall_lengths_conf.append(st.norm.interval(alpha = conf_interval, loc = avg_lengths, scale = st.sem(seglen_aggregate_stall_lengths)))
    stdev_stall_lengths.append(np.std(seglen_aggregate_stall_lengths))

: 

### Stall lengths

In [ ]:
avg_stall_lengths

: 

In [ ]:
avg_stall_lengths_conf

: 

In [ ]:
stdev_stall_lengths

: 

### Stall counts

In [ ]:
avg_stall_cts

: 

In [ ]:
avg_stall_cts_conf

: 

In [ ]:
stdev_stall_cts

: 

In [ ]:
import numpy as np
stalls_permin = np.divide(avg_stall_cts, video_length_min)
print("Average stalls per minute of video (5s, 10s, 20s): {stallavg}".format(stallavg = stalls_permin))

: 

### Stall totals

In [ ]:
import numpy as np
stall_totals = np.multiply(avg_stall_cts, avg_stall_lengths)
print("Stall totals (# of stalls * length of stalls) (sec):\n{vols}".format(vols = stall_totals / 1000))

: 

In [ ]:
import numpy as np
stall_totals_permin = np.divide(stall_totals, video_length_min)
print("Average stall totals per minute of video (seconds of stall / minute of video):\n{avgvol}".format(avgvol = stall_totals_permin / 1000))

: 

## Quality Changes

In [ ]:
log_res_pattern = re.compile("^LOG  [^,]+,[^,]+,([\dx]+).+$")
qual_change_pattern = re.compile("^QUAL  at [\d\.]+ to ([\dx]+)$")

def ComputeQualChangeData(dir_path):
    jsLog_names = [filename for filename in os.listdir(dir_path) if filename.startswith("Tester_") and filename.endswith(".log")]
    if len(jsLog_names) != 1:
        return
    jsLog_name = jsLog_names[0]
    jsLog = open(os.path.join(dir_path, jsLog_name), "r")
    
    startRes = None
    resAmnts = dict()
    
    # Read through all the lines.
    while line := jsLog.readline():
        log_res_match = log_res_pattern.match(line)
        if log_res_match and startRes == None:
            # This is the first LOG line. Let's get the quality level in case it never changes so there are no QUAL entries.
            startRes = log_res_match.group(1)
            resAmnts[startRes] = resAmnts.get(startRes, 0) + 1
        
        match = qual_change_pattern.match(line)
        if match:
            resolution = match.group(1)
            resAmnts[resolution] = resAmnts.get(resolution, 0) + 1 # Increase number of occurrences of this resolution by 1.
    
    return resAmnts

: 

In [ ]:
seg_lens = ["5s", "10s", "20s"]
avg_qual_changes = list()
stdev_qual_changes = list()
for seg_len in seg_lens:
    #print("Segment length: " + seg_len)
    dir_path = "./{seg_len}".format(seg_len = seg_len)
    
    #run_ct = 0
    cur_qual_changes = list()
    #sum_qual_changes = 0
    
    for file_name in os.listdir(dir_path):
    #for run_idx in range(0, 3): # Index from [0..2].
        file_path = os.path.join(dir_path, file_name)
        if not os.path.isdir(file_path): # Only looking to read directories (that are symlinks).
            continue
        resAmnts = ComputeQualChangeData(file_path)
        
        #print(resAmnts)
        #run_ct += 1
        # Subtract one because the video always starts at a certain quality, that's not a quality change.
        qual_changes = sum(resAmnts.values()) - 1
        cur_qual_changes.append(qual_changes)
        #sum_qual_changes += qual_changes
        #seg_stall_ct_sum += stall_ct
        #seg_stall_len_sum += avg_stall_len_ms
        #print("Stall count: " + str(stall_ct))
        #print("Average stall length (ms): " + str(avg_stall_len_ms))
    
    avg_qual_changes.append(np.mean(cur_qual_changes))
    stdev_qual_changes.append(np.std(cur_qual_changes))
    #avg_qual_changes.append(sum_qual_changes / run_ct)
    #avg_stall_cts.append(seg_stall_ct_sum / run_ct)
    #avg_stall_lengths.append(seg_stall_len_sum / run_ct)

: 

In [ ]:
avg_qual_changes

: 

In [ ]:
stdev_qual_changes

: 

## Quality Data

In [ ]:
log_res_pattern = re.compile("^LOG  [^,]+,[^,]+,([\dx]+).+$")

def ComputeQualData(dir_path):
    jsLog_names = [filename for filename in os.listdir(dir_path) if filename.startswith("Tester_") and filename.endswith(".log")]
    #print(jsLog_names)
    if len(jsLog_names) != 1:
        return
    jsLog_name = jsLog_names[0]
    jsLog = open(os.path.join(dir_path, jsLog_name), "r")
    
    resAmnts = dict()
    resOccurrences = list()
    
    # Read through all the lines.
    while line := jsLog.readline():
        log_res_match = log_res_pattern.match(line)
        if not log_res_match:
            continue

        startRes = log_res_match.group(1)
        #print(startRes)
        resAmnts[startRes] = resAmnts.get(startRes, 0) + 1
        resOccurrences.append(startRes)
        
    return resOccurrences

: 

In [ ]:
seg_lens = ["5s", "10s", "20s"]
avg_stall_cts = list()
avg_stall_lengths = list()
resOccurrences = list()
for seg_len in seg_lens:
    print("Segment length: " + seg_len)
    dir_path = "./{seg_len}".format(seg_len = seg_len)
    
    run_ct = 0
    seg_stall_ct_sum = 0
    seg_stall_len_sum = 0
    resQtys = list()
    
    for file_name in os.listdir(dir_path):
    #for run_idx in range(0, 3): # Index from [0..2].
        file_path = os.path.join(dir_path, file_name)
        if not os.path.isdir(file_path): # Only looking to read directories (that are symlinks).
            continue
        #resAmnts = ComputeQualData(file_path)
        resQtys += ComputeQualData(file_path)
        #print(resAmnts)
        #run_ct += 1
        #seg_stall_ct_sum += stall_ct
        #seg_stall_len_sum += avg_stall_len_ms
        #print("Stall count: " + str(stall_ct))
        #print("Average stall length (ms): " + str(avg_stall_len_ms))
    
    resOccurrences.append(resQtys)
    #avg_stall_cts.append(seg_stall_ct_sum / run_ct)
    #avg_stall_lengths.append(seg_stall_len_sum / run_ct)

: 

In [ ]:
len(resOccurrences[2])

: 

In [ ]:
from collections import Counter
import pandas as pd
resAmnts = list()
#resAmnts = pd.DataFrame()
for i in range(0, len(resOccurrences)):
    occDict = dict(Counter(resOccurrences[i]))
    occurrenceFrame = pd.DataFrame()
    occurrenceFrame["Resolution"] = occDict.keys()
    occurrenceFrame["Occurrences"] = occDict.values()
    totalOccurrences = occurrenceFrame["Occurrences"].sum()
    occurrenceFrame["Proportion"] = occurrenceFrame["Occurrences"] / totalOccurrences
    resAmnts.append(occurrenceFrame)

: 

In [ ]:
resAmnts[0]

: 

In [ ]:
resAmnts[0].loc[resAmnts[0]["Occurrences"].idxmax()]
#resAmnts[0].loc[Occurrences == resAmnts[0]["Occurrences"]

: 

In [ ]:
resAmnts[1]

: 

In [ ]:
resAmnts[2]

: 

In [ ]:
import plotly.express as px

fig = px.ecdf(resOccurrences[0], ecdfnorm = "percent")
#fig.data
#fig.data[0].line.color = "red"
#fig.data[1].line.color = "green"
#fig.data[2].line.color = "blue"
#fig.update_xaxes(categoryorder = "array", categoryarray = ["480x270", "640x360", "960x540", "1280x720", "1920x1080", "3840x2160"])
fig.update_xaxes(title = "Resolution")
fig.show()

: 

## RTT Time

In [ ]:
import os
import csv
import numpy as np

conf_interval = 0.95

def calculate_rtt(folder_path):
    folder_path = folder_path
    #run_folders = ["Run0", "Run1", "Run2"]
    #run_folders = [filename for filename in os.listdir(dir_path) if filename.startswith("Tester_") and filename.endswith(".log")]
    rtt_list = []

    for run_folder in run_folders:
        file_path = os.path.join(folder_path, run_folder, "UDPing_log.csv")
        with open(file_path, "r") as csv_file:
            csv_reader = csv.reader(csv_file)
            next(csv_reader) # skip "Pinging..." row
            next(csv_reader) # skip "empty" row
            next(csv_reader) # skip header row
            for row in csv_reader:
                if "Ping statistics" in row[0]:
                    break
                rtt_list.append(float(row[2]))

    mean_rtt = np.mean(rtt_list)
    mean_rtt_conf = st.norm.interval(alpha = conf_interval, loc = mean_rtt, scale = st.sem(rtt_list))
    std_rtt = np.std(rtt_list)
    #avg_stall_cts_conf.append(st.norm.interval(alpha = conf_interval, loc = avg_cts, scale = st.sem(seglen_aggregate_stall_counts)))

    return (mean_rtt, mean_rtt_conf, std_rtt)

mean_rtt, mean_rtt_conf, std_rtt = calculate_rtt("./10s")
print("Mean RTT:", mean_rtt)
print("Mean RTT confidence interval:", mean_rtt_conf)
print("Standard deviation of RTT:", std_rtt)

: 

In [ ]:
import os
import csv
import numpy as np

def calculate_rtt(folder_path):
    folder_path = folder_path
    #run_folders = ["Run0", "Run1", "Run2"]
    run_folders = ["Results_2023-02-27T00_23_03.144711_glomma", "Results_2023-02-27T01_58_06.647754_glomma", "Results_2023-02-27T02_16_01.406052_glomma"]
    packet_loss_seqs=[]
    rtt_list = []
    receive_rates =[]
    prev_seq=-9999

    for run_folder in run_folders:
        file_path = os.path.join(folder_path, run_folder, "UDPing_log.csv")
        with open(file_path, "r") as csv_file:
            csv_reader = csv.reader(csv_file)
            next(csv_reader) # skip "Pinging..." row
            next(csv_reader) # skip "empty" row
            next(csv_reader) # skip header row
            for row in csv_reader:
                if "Ping statistics" in row[0]:
                    break
                rtt = float(row[2])
                rtt_list.append(rtt)

                seq = int(row[3])
                if prev_seq != -9999 and seq != prev_seq + 1:
                    packet_loss_seqs.append(seq)
                prev_seq = seq


            send=int(csv_reader.__next__()[1])
            receive=int(csv_reader.__next__()[1])
            if receive == 0:
                receive_rates.append(0) # There are cases that all packets got lost.
            else:
                receive_rates.append(send/receive)

    mean_rtt = np.mean(rtt_list)
    std_rtt = np.std(rtt_list)
    mean_receive_rate=np.mean(receive_rates)
    std_receive_rate=np.std(receive_rates)

    return (mean_rtt, std_rtt,mean_receive_rate,std_receive_rate, packet_loss_seqs)

mean_rtt, std_rtt, mean_receive_rate, std_receive_rate, packet_loss_seqs= calculate_rtt("./5s")
print("Mean RTT:", mean_rtt)
print("Standard deviation of RTT:", std_rtt)
print("Mean Receive rate:", mean_receive_rate)
print("Standard deviation of Receive rate:", std_receive_rate)
print("Packet loss sequences:", packet_loss_seqs)

: 

In [ ]:
import os
import csv
import numpy as np

def calculate_rtt(folder_path):
    folder_path = folder_path
    #run_folders = ["Run0", "Run1", "Run2"]
    run_folders = ["Results_2023-02-27T00_23_03.144711_glomma", "Results_2023-02-27T01_58_06.647754_glomma", "Results_2023-02-27T02_16_01.406052_glomma"]
    all_packet_loss_seqs=[]
    rtt_list = []
    receive_rates =[]

    for run_folder in run_folders:
        prev_seq=-9999
        packet_loss_seqs=[]
        file_path = os.path.join(folder_path, run_folder, "UDPing_log.csv")
        with open(file_path, "r") as csv_file:
            csv_reader = csv.reader(csv_file)
            next(csv_reader) # skip "Pinging..." row
            next(csv_reader) # skip "empty" row
            next(csv_reader) # skip header row
            for row in csv_reader:
                if "Ping statistics" in row[0]:
                    break
                rtt = float(row[2])
                rtt_list.append(rtt)

                seq = int(row[3])
                if prev_seq != -9999 and seq != prev_seq + 1:
                    packet_loss_seqs.append(seq)
                prev_seq = seq


            all_packet_loss_seqs.append(packet_loss_seqs)
            send=int(csv_reader.__next__()[1])
            receive=int(csv_reader.__next__()[1])
            if receive == 0:
                receive_rates.append(0) # There are cases that all packets got lost.
            else:
                receive_rates.append(send/receive)


    mean_rtt = np.mean(rtt_list)
    std_rtt = np.std(rtt_list)
    mean_receive_rate=np.mean(receive_rates)
    std_receive_rate=np.std(receive_rates)

    return (mean_rtt, std_rtt,mean_receive_rate,std_receive_rate, all_packet_loss_seqs)

mean_rtt, std_rtt, mean_receive_rate, std_receive_rate, packet_loss_seqs= calculate_rtt("./5s")
print("Mean RTT:", mean_rtt)
print("Standard deviation of RTT:", std_rtt)
print("Mean Receive rate:", mean_receive_rate)
print("Standard deviation of Receive rate:", std_receive_rate)
print("Packet loss sequences:", packet_loss_seqs)

: 

In [ ]:
import os
import csv
import numpy as np

def calculate_rtt(folder_path):
    folder_path = folder_path
    #run_folders = ["Run0", "Run1", "Run2"]
    run_folders = ["Results_2023-02-27T00_23_03.144711_glomma", "Results_2023-02-27T01_58_06.647754_glomma", "Results_2023-02-27T02_16_01.406052_glomma"]
    all_packet_loss_seqs=[]
    rtt_list = []
    receive_rates =[]

    for run_folder in run_folders:
        file_path = os.path.join(folder_path, run_folder, "UDPing_log.csv")
        with open(file_path, "r") as csv_file:
            csv_reader = csv.reader(csv_file)
            next(csv_reader) # skip "Pinging..." row
            next(csv_reader) # skip "empty" row
            next(csv_reader) # skip header row
            for row in csv_reader:
                if "Ping statistics" in row[0]:
                    break
                rtt = float(row[2])
                rtt_list.append(rtt)

            send=int(csv_reader.__next__()[1])
            receive=int(csv_reader.__next__()[1])
            if receive == 0:
                receive_rates.append(0) # There are cases that all packets got lost.
            else:
                receive_rates.append(send/receive)

        with open(file_path, "r") as csv_file:
            csv_reader = csv.reader(csv_file)
            prev_seq=-9999
            packet_loss_seqs=[]
            next(csv_reader)  # skip "Pinging..." row
            next(csv_reader)  # skip "empty" row
            next(csv_reader)  # skip header row
            for row in csv_reader:
                if "Ping statistics" in row[0]:
                    break

                seq = int(row[3])
                if prev_seq != -9999 and seq != prev_seq + 1:
                    packet_loss_seqs.append(seq)
                prev_seq = seq

            print(packet_loss_seqs)
            all_packet_loss_seqs.append(packet_loss_seqs)


    mean_rtt = np.mean(rtt_list)
    std_rtt = np.std(rtt_list)
    mean_receive_rate=np.mean(receive_rates)
    std_receive_rate=np.std(receive_rates)

    return (mean_rtt, std_rtt,mean_receive_rate,std_receive_rate, all_packet_loss_seqs)
mean_rtt, std_rtt, mean_receive_rate, std_receive_rate, packet_loss_seqs= calculate_rtt("./5s")
print("Mean RTT:", mean_rtt)
print("Standard deviation of RTT:", std_rtt)
print("Mean Receive rate:", mean_receive_rate)
print("Standard deviation of Receive rate:", std_receive_rate)
print("Packet loss sequences:", packet_loss_seqs)

: 

## Time to Start Playing

In [ ]:
import os
import re

log_video_time_pattern = re.compile("^LOG  ([\d\.]+),([\d\.]+):.+$")
#stall_pattern = re.compile("^STALL  ([\d]+) ms and stopped at [\d\.]+ sec", re.M)

def ComputeTimeToStart(dir_path):
    jsLog_names = [filename for filename in os.listdir(dir_path) if filename.startswith("Tester_") and filename.endswith(".log")]
    if len(jsLog_names) != 1:
        return
    jsLog_name = jsLog_names[0]
    jsLog = open(os.path.join(dir_path, jsLog_name), "r")
    
    # Read through all the lines.
    while line := jsLog.readline():
        match = log_video_time_pattern.match(line)
        if match:
            absolute_time = float(match.group(1))
            video_time = float(match.group(2))
            if video_time > 0:
                # We've found when the video started playing.
                return absolute_time
    
    return -1

: 

In [ ]:
import numpy as np
import scipy.stats as st

conf_interval = 0.95
seg_lens = ["5s", "10s", "20s"]

avg_start_times_sec = list()
avg_start_times_sec_conf = list()
stdev_start_times_sec = list()

for seg_len in seg_lens:
    dir_path = "./{seg_len}".format(seg_len = seg_len)
    
    start_times_sec = list()
    
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)
        if not os.path.isdir(file_path): # Only looking to read directories (that are symlinks).
            continue
        start_time_sec = ComputeTimeToStart(file_path)
        start_times_sec.append(start_time_sec)
    
    avg_start_time_sec = np.mean(start_times_sec)
    avg_start_times_sec.append(avg_start_time_sec)
    #avg_start_times_sec_conf.append(st.norm.interval(alpha = conf_interval, loc = avg_start_time_sec, scale = st.sem(start_times_sec)))
    avg_start_times_sec_conf.append(st.t.interval(alpha = conf_interval,
                                                  df = len(start_times_sec) - 1,
                                                  loc = avg_start_time_sec,
                                                  scale = st.sem(start_times_sec)))
    stdev_start_times_sec.append(np.std(start_times_sec))

: 

In [ ]:
avg_start_times_sec

: 

In [ ]:
avg_start_times_sec_conf

: 

In [ ]:
stdev_start_times_sec

: 

: 